<a href="https://colab.research.google.com/github/dhaval0122/cd12380-data-pipelines-with-airflow/blob/main/RouteData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install prophet openpyxl --quiet

In [ ]:
# STEP 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# STEP 3: Load the Excel File
import pandas as pd
import os
# Replace with your actual path
file_path = '/content/drive/My Drive/ML_FORECAST/Route_Data/Location_details - Copy.xlsx'
# Check if the file exists
if os.path.exists(file_path):
    print("✅ File found. Reading now...")
    df = pd.read_excel(file_path)
else:
    print("❌ File not found. Please check the path.")

✅ File found. Reading now...


In [ ]:
# Load the Excel file
df = pd.read_excel(file_path, engine='openpyxl')
# Show top 5 rows
df.head()

,Master,DG_NDG,Mode of TP,Origin City,Origin Country Code,Equipment type,Destination City,Cost per load,Currency,Remark,Departure Port,Destination Port,Custom_Packing_Material,Origine_Country_New,Destination_Country_New,Destination Country Code
0,Road freight,DG,Export,Jinshan,CN,NaN,Shanghai,2426,CNY,NaN,NaN,NaN,20 ft Box - Reefer operated,China,China,CN
1,Road freight,NDG,Export,Jinshan,CN,NaN,Shanghai,1662,CNY,NaN,NaN,NaN,20 ft Box - Reefer operated,China,China,CN
2,Road freight,DG,Export,Pudong,CN,NaN,Shanghai,2482,CNY,NaN,NaN,NaN,20 ft Box - Reefer operated,China,China,CN
3,Road freight,NDG,Export,Pudong,CN,NaN,Shanghai,2307,CNY,NaN,NaN,NaN,20 ft Box - Reefer operated,China,China,CN
4,Road freight,DG,Export,Waigaoqiao Free Trade Zone,CN,NaN,Shanghai,2255,CNY,NaN,NaN,NaN,20 ft Box - Reefer operated,China,China,CN


In [ ]:
# Create a copy of the original dataframe for safety
df_cleaned = df.copy()

columns_to_clean = [
    'Origin City', 'Origine_Country_New', 'Destination City',
    'Destination_Country_New', 'Departure Port', 'Destination Port',
    'Mode of TP', 'DG_NDG'
]

for col in columns_to_clean:
    df_cleaned[col] = df_cleaned[col].astype(str).str.strip().str.lower()

# Preview the cleaned data
df_cleaned.head()


,Master,DG_NDG,Mode of TP,Origin City,Origin Country Code,Equipment type,Destination City,Cost per load,Currency,Remark,Departure Port,Destination Port,Custom_Packing_Material,Origine_Country_New,Destination_Country_New,Destination Country Code
0,Road freight,dg,export,jinshan,CN,NaN,shanghai,2426,CNY,NaN,nan,nan,20 ft Box - Reefer operated,china,china,CN
1,Road freight,ndg,export,jinshan,CN,NaN,shanghai,1662,CNY,NaN,nan,nan,20 ft Box - Reefer operated,china,china,CN
2,Road freight,dg,export,pudong,CN,NaN,shanghai,2482,CNY,NaN,nan,nan,20 ft Box - Reefer operated,china,china,CN
3,Road freight,ndg,export,pudong,CN,NaN,shanghai,2307,CNY,NaN,nan,nan,20 ft Box - Reefer operated,china,china,CN
4,Road freight,dg,export,waigaoqiao free trade zone,CN,NaN,shanghai,2255,CNY,NaN,nan,nan,20 ft Box - Reefer operated,china,china,CN


In [ ]:
# Define the updated route finder logic
def find_route_by_country_city(df, origin_city, origin_country, destination_city, destination_country):
    origin_city = origin_city.lower().strip()
    origin_country = origin_country.lower().strip()
    destination_city = destination_city.lower().strip()
    destination_country = destination_country.lower().strip()

    # Leg 1: From origin (Export or Road)
    leg1 = df[
        (df['Origin City'] == origin_city) &
        (df['Origine_Country_New'] == origin_country) &
        (df['Mode of TP'].isin(['export', 'road']))
    ]

    # Leg 2: To destination (Sea)
    leg2 = df[
        (df['Destination City'] == destination_city) &
        (df['Destination_Country_New'] == destination_country) &
        (df['Mode of TP'] == 'sea')
    ]

    # Merge leg1 and leg2 where leg1 destination city matches leg2 origin city or departure port
    merged = pd.merge(
        leg1,
        leg2,
        left_on='Destination City',
        right_on='Origin City',
        suffixes=('_leg1', '_leg2')
    )

    if merged.empty:
        merged = pd.merge(
            leg1,
            leg2,
            left_on='Destination City',
            right_on='Departure Port',
            suffixes=('_leg1', '_leg2')
        )

    return merged


In [ ]:
# Run the route search for Jinshan (China) → Jakarta (Indonesia)
route_result = find_route_by_country_city(
    df_cleaned,
    origin_city="Jinshan",
    origin_country="China",
    destination_city="Jakarta",
    destination_country="Indonesia"
)

display(route_result)

,Master_leg1,DG_NDG_leg1,Mode of TP_leg1,Origin City_leg1,Origin Country Code_leg1,Equipment type_leg1,Destination City_leg1,Cost per load_leg1,Currency_leg1,Remark_leg1,...,Destination City_leg2,Cost per load_leg2,Currency_leg2,Remark_leg2,Departure Port_leg2,Destination Port_leg2,Custom_Packing_Material_leg2,Origine_Country_New_leg2,Destination_Country_New_leg2,Destination Country Code_leg2
0,Road freight,dg,export,jinshan,CN,NaN,shanghai,2426,CNY,NaN,...,jakarta,615,USD,NaN,shanghai (31),jakarta (java),20 ft Box - Reefer operated,china,indonesia,ID
1,Road freight,ndg,export,jinshan,CN,NaN,shanghai,1662,CNY,NaN,...,jakarta,615,USD,NaN,shanghai (31),jakarta (java),20 ft Box - Reefer operated,china,indonesia,ID


In [ ]:
multi_leg_city_based = find_route_by_country_city(
    df_cleaned,
    origin_city="Pudong",
    origin_country="China",
    destination_city="Jakarta",
    destination_country="Indonesia"
)

display(multi_leg_city_based)

,Master_leg1,DG_NDG_leg1,Mode of TP_leg1,Origin City_leg1,Origin Country Code_leg1,Equipment type_leg1,Destination City_leg1,Cost per load_leg1,Currency_leg1,Remark_leg1,...,Destination City_leg2,Cost per load_leg2,Currency_leg2,Remark_leg2,Departure Port_leg2,Destination Port_leg2,Custom_Packing_Material_leg2,Origine_Country_New_leg2,Destination_Country_New_leg2,Destination Country Code_leg2
0,Road freight,dg,export,pudong,CN,NaN,shanghai,2482,CNY,NaN,...,jakarta,615,USD,NaN,shanghai (31),jakarta (java),20 ft Box - Reefer operated,china,indonesia,ID
1,Road freight,ndg,export,pudong,CN,NaN,shanghai,2307,CNY,NaN,...,jakarta,615,USD,NaN,shanghai (31),jakarta (java),20 ft Box - Reefer operated,china,indonesia,ID


In [ ]:
multi_leg_city_based1 = find_route_by_country_city(
    df_cleaned,
    origin_city="Waigaoqiao Free Trade Zone",
    origin_country="China",
    destination_city="Jakarta",
    destination_country="Indonesia"
)

display(multi_leg_city_based1)

,Master_leg1,DG_NDG_leg1,Mode of TP_leg1,Origin City_leg1,Origin Country Code_leg1,Equipment type_leg1,Destination City_leg1,Cost per load_leg1,Currency_leg1,Remark_leg1,...,Destination City_leg2,Cost per load_leg2,Currency_leg2,Remark_leg2,Departure Port_leg2,Destination Port_leg2,Custom_Packing_Material_leg2,Origine_Country_New_leg2,Destination_Country_New_leg2,Destination Country Code_leg2
0,Road freight,dg,export,waigaoqiao free trade zone,CN,NaN,shanghai,2255,CNY,NaN,...,jakarta,615,USD,NaN,shanghai (31),jakarta (java),20 ft Box - Reefer operated,china,indonesia,ID
1,Road freight,ndg,export,waigaoqiao free trade zone,CN,NaN,shanghai,1719,CNY,NaN,...,jakarta,615,USD,NaN,shanghai (31),jakarta (java),20 ft Box - Reefer operated,china,indonesia,ID
